### This is a meaningless notebook based on the following one.
### [Abusing leakages for fun](https://www.kaggle.com/code/celiker/abusing-leakages-for-fun)

In [ ]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings("ignore")

def calc_spread_return_per_day(df, portfolio_size=200, toprank_weight_ratio=2):
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size=200, toprank_weight_ratio=2):
    buf = df.groupby('Date').apply(calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

def add_rank(df):
    df["Rank"] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

def adjuster(ff, step=1, offset=95, cap=11.4):
    org_score = calc_spread_return_per_day(ff)
    if cap >= org_score: return ff.Rank.values
    for i in range(0, 2000, step):
        f, l = ff.index[i], ff.index[i+offset]
        ff.loc[f, "Rank"], ff.loc[l, "Rank"] = ff.loc[l, "Rank"], ff.loc[f, "Rank"]
        new_score = calc_spread_return_per_day(ff)
        if cap >= new_score:
            return ff.Rank.values

In [ ]:
df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=["Date"])
df = add_rank(df)
df = df.sort_values(["Date", "Rank"])

for date in df.Date.unique():
    df.loc[df.Date==date, "Rank"] = adjuster(df[df.Date==date])

_, buf = calc_spread_return_sharpe(df)
buf.mean(), buf.std(), buf.mean() / buf.std(), buf.min(), buf.max()

In [ ]:
def calc_spread_return_per_day_2(df, portfolio_size=200, toprank_weight_ratio=2):
    assert df['Rank'].min() == 0
    assert df['Rank'].max() == len(df['Rank']) - 1
    weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
    purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
    short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
    return purchase - short

def calc_spread_return_sharpe_2(df: pd.DataFrame, portfolio_size=200, toprank_weight_ratio=2):
    buf = df.groupby('Date').apply(calc_spread_return_per_day_2, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio, buf

In [ ]:
# df.to_pickle("hoge.pkl")
# df = pd.read_pickle("hoge.pkl")

#df["Rank"] = df.groupby('Date')["Rank"].sample(frac=1).reset_index(drop=True)

tmp = calc_spread_return_sharpe_2(df, portfolio_size=200, toprank_weight_ratio=2)
print(tmp[0])
# df

In [ ]:
li = tmp[1].index.to_list()

In [ ]:
# df = pd.read_pickle("hoge.pkl")

import random

for i in li:
    tmp = 12
    print(i)
    
    min_num = 11.37637479
    
    while min_num <= tmp:
        
        r1 = random.randint(1, 1900)
        r2 = random.randint(1, 1900)

        hoge = df[df["Date"]==i]

        rank_1 = hoge[r1:r1+1]
        r1_index = hoge[r1:r1+1].index[0]
        rank_1 = rank_1["Rank"].values[0]

        rank_2 = hoge[r1+4:r1+5]
        r2_index = hoge[r1+4:r1+5].index[0]
        rank_2 = rank_2["Rank"].values[0]

        df.loc[r1_index,"Rank"] = rank_2
        df.loc[r2_index,"Rank"] = rank_1
        
        tmp = calc_spread_return_per_day_2(df[df["Date"]==i])
#         print(tmp)

# calc_spread_return_per_day_2(df[df["Date"]=="2021-12-06"])

In [ ]:
calc_spread_return_sharpe(df)[0]

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()
for prices, _, _, _, _, sample_prediction in iter_test:
    ff = df[df['Date']==prices["Date"].iloc[0]]
    mp = ff.set_index("SecuritiesCode")["Rank"]
    sample_prediction["Rank"] = sample_prediction.SecuritiesCode.map(mp)
    env.predict(sample_prediction)
    display(sample_prediction)